## SQL & BQ
Collection of files queried from or pushed to BQ

- All validation outputs goes to OutputTD/6_validation
- All validation data goes to DataTD/validation
- Naming: `6.X_name_R.ipynb` = folder's order.file's order.filename_Rcode.
- Under filename, `coh2` means the output cohort used `cohort2` as the cohort input
- Use shc_core_2021

NOTES:
- num_value1 --> numerical_val_1
- num_value2 --> numerical_val_2
- diagnosis_code --> diagnosis

In [1]:
# !pip install pandas-gbq

In [2]:
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# %matplotlib inline
# %load_ext rpy2.ipython

In [3]:
import os 
from google.cloud import bigquery
from google.cloud.bigquery import dbapi

##Use correct path based on whether you are, Nero or local
# use Ctrl + Insert to copy and Shift + Insert to paste

# for Nero:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/minh084/.config/gcloud/application_default_credentials.json' 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter/.config/gcloud/application_default_credentials.json'

# for local computer:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\AppData\Roaming\gcloud\application_default_credentials.json' 

##set correct Nero project
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 

##Setting up BQ API, m1:
client = bigquery.Client()

##using dbAPI connection, m2:
conn = dbapi.connect(client)

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### ADT check service, class to get admitted ED patients

In [4]:
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Run 6.1_cohort_val notebook
Results in `6_1_cohort1` as the original validation cohort.

Label is the label for highest level of care within 24 hours since admission.

### Queries information for other inclusion/ex criteria
When querying, join with `6_1_cohort1`
- Inpatient/hospital encounters only
- Full code only
- Age 18 and older only

In [6]:
# m1_encounter.sql

q = """ 

SELECT c.*,
    e.inpatient_data_id_coded, 
    e.enc_type, e.visit_type, e.acuity_level, e.ACUITY_LEVEL_C,
    e.hosp_admsn_time_jittered_utc
FROM 
    `som-nero-phi-jonc101.shc_core_2021.encounter` as e
RIGHT JOIN 
    `som-nero-phi-jonc101.triageTD.6_1_cohort1` as c
ON (c.anon_id=e.anon_id and c.pat_enc_csn_id_coded=e.pat_enc_csn_id_coded)
ORDER BY
  c.anon_id
  
"""

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'encounters_2021.csv'), index=False)

In [7]:
# m2_codestatus.sql

q = """ 

SELECT c.*,
    o.order_type, o.order_status, o.display_name, o.description, 
    o.order_time_jittered_utc
FROM 
    `som-nero-phi-jonc101.shc_core_2021.order_proc` as o
JOIN 
    `som-nero-phi-jonc101.triageTD.6_1_cohort1` as c
ON (c.anon_id=o.anon_id and c.pat_enc_csn_id_coded=o.pat_enc_csn_id_coded)
WHERE o.order_type = "Code Status"
ORDER BY
  c.anon_id
""" 

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'code_status_2021.csv'), index=False)

In [8]:
# m3_demographic.sql
# there is NO d.recent_conf_enc_jittered as recent_date,

q = """ 

SELECT c.anon_id,
    d.gender, d.canonical_race as race, d.language, 
    d.recent_ht_in_cms as recent_height_cm, d.recent_wt_in_kgs as recent_weight_kg,
    d.insurance_payor_name as insurance,  
    DATE(CAST(d.birth_date_jittered as TIMESTAMP)) as dob
FROM 
    `som-nero-phi-jonc101.shc_core_2021.demographic` as d
JOIN 
    `som-nero-phi-jonc101.triageTD.6_1_cohort1` as c
ON c.anon_id=d.anon_id
ORDER BY
  c.anon_id
"""

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'demographics_2021.csv'), index=False)

### Pushed the 6_2_cohort2 after running 6.2_criteria_R notebook
Use `6_1_cohort1` to filter out patients with exclusion/inc criteria, resulting in updated `6_2_cohort2`

In [9]:
df = pd.read_csv(os.path.join(valdir, "6_2_cohort2.csv"))
print(len(df)) # 17128
print(list(df.columns))

17128
['anon_id', 'pat_enc_csn_id_coded', 'admit_time_jittered', 'label', 'admit_time', 'adm_year', 'adm_month', 'inpatient_data_id_coded', 'ESI', 'hosp_admsn_time', 'ed_time_hr', 'gender', 'race', 'language', 'recent_height_cm', 'recent_weight_kg', 'insurance', 'age']


In [10]:
df.head()

,anon_id,pat_enc_csn_id_coded,admit_time_jittered,label,admit_time,adm_year,adm_month,inpatient_data_id_coded,ESI,hosp_admsn_time,ed_time_hr,gender,race,language,recent_height_cm,recent_weight_kg,insurance,age
0,JC1000116,131295313275,2020-09-29 22:45:00+00:00,0,2020-09-29 22:45:00,2020,9,57868578,3.0,2020-09-29 16:02:00,6.716667,Female,Other,Spanish,154.00,73.05,HPSM,44
1,JC1000939,131295018112,2020-08-22 11:41:00+00:00,1,2020-08-22 11:41:00,2020,8,57698292,2.0,2020-08-22 07:47:00,3.900000,Female,Other,Spanish,154.94,136.80,HPSM,49
2,JC1001580,131300665918,2020-12-29 04:15:00+00:00,0,2020-12-29 04:15:00,2020,12,61108864,3.0,2020-12-29 02:58:00,1.283333,Male,White,English,187.96,89.12,UNITED HEALTHCARE,54
3,JC1001688,131288774622,2020-06-19 21:36:00+00:00,0,2020-06-19 21:36:00,2020,6,53810016,3.0,2020-06-19 19:45:00,1.850000,Male,Asian,English,205.74,163.29,ALAMEDA ALLIANCE MCAL MGD CARE,37
4,JC1001688,131302440087,2021-02-26 11:46:00+00:00,0,2021-02-26 11:46:00,2021,2,62198093,3.0,2021-02-26 09:12:00,2.566667,Male,Asian,English,205.74,163.29,ALAMEDA ALLIANCE MCAL MGD CARE,37


In [11]:
# removed {'name' : 'recent_date', 'type' : 'DATE'}
table_schema = [{'name' : 'anon_id', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'inpatient_data_id_coded', 'type': 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'label', 'type' : 'INTEGER'},
                {'name' : 'hosp_admsn_time', 'type': 'TIMESTAMP'},
                {'name' : 'ed_time_hr', 'type': 'FLOAT'},
                {'name' : 'ESI', 'type': 'INTEGER'},
                {'name' : 'age', 'type' : 'INTEGER'},
                {'name' : 'gender', 'type' : 'STRING'},
                {'name' : 'race', 'type' : 'STRING'},
                {'name' : 'language', 'type' : 'STRING'},
                {'name' : 'insurance', 'type' : 'STRING'},
                {'name' : 'recent_height_cm', 'type' : 'FLOAT'},
                {'name' : 'recent_weight_kg', 'type' : 'FLOAT'}]
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_2_cohort2'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
                 project_id='som-nero-phi-jonc101',
                 table_schema=table_schema,
                 if_exists='replace')

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:04,  4.36s/it]


### Use updated 6_2_cohort2 with inpatient_id_coded to query flowsheet

In [12]:
# m4_HWflowsheet.sql

q = """ 

SELECT c.anon_id, c.pat_enc_csn_id_coded, c.inpatient_data_id_coded,
    f.row_disp_name, f.units, f.recorded_time_utc, f.numerical_val_1 as num_value1, f.numerical_val_2 as num_value2
FROM 
    `som-nero-phi-jonc101.triageTD.6_2_cohort2` as c
JOIN 
    `som-nero-phi-jonc101.shc_core_2021.flowsheet` as f
ON 
    (c.anon_id=f.anon_id and c.inpatient_data_id_coded=f.inpatient_data_id_coded)
WHERE
(
(row_disp_name="Weight")
OR
(row_disp_name="Height")
)
"""

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'HWflowsheet_2021.csv'), index=False)

In [13]:
# m5_flowsheet.sql

q = """ 

SELECT c.anon_id, c.pat_enc_csn_id_coded, c.inpatient_data_id_coded, c.admit_time, c.label,
    f.template, f.row_disp_name, f.units, f.recorded_time_utc, f.numerical_val_1 as num_value1, f.numerical_val_2 as num_value2
FROM 
    `som-nero-phi-jonc101.triageTD.6_2_cohort2` as c 
JOIN 
    `som-nero-phi-jonc101.shc_core_2021.flowsheet` as f
ON 
    (c.anon_id=f.anon_id and c.inpatient_data_id_coded=f.inpatient_data_id_coded)
WHERE
    recorded_time_utc < admit_time --, 'yyyy-mm-dd hh24:mi:ss'
AND row_disp_name in 
('Heart Rate', 'Pulse', "Resting HR", 'Resting Heart Rate (bpm)', 'Resting Pulse Rate: (Record BPM)', -- smaller number, might be too noisy
 'O2', 'O2 (LPM)', 'O2 Flow (L/min)', 'O2 Delivery Method', 
 'Resp Rate', 'Resp', 'Respiratory Rate', -- "Resting RR" not there
 'BP', 'NIBP', 'Arterial Systolic BP' , 'Arterial Diastolic BP' , 'Blood Pressure', "Resting BP", --'Resting Systolic Blood Pressure',
 'Temp', 'Temp (in Celsius)', 'Temperature (Blood - PA line)', 'Temp 2', 'Temperature', 
 'Activity', 'Mobility', 
 'acuity score', 'Acuity as Level of Care',
 'LOC', 'LOC Score')
-- removed GCS, too many missing and not consistent
--  'SpO2', "Resting SpO2", 'Oxygen Saturation', 'Resting O2 Saturation', -- difficult to interpret without O2 delivery
"""

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'flowsheet_2021.csv'), index=False)

### Push 6_3_cohort3 file from R notebook to Big Query
- From 6.5_cohort3_vitalsigns_R.ipynb
- This is the updated cohort with at least a complete set of vital signs. First set in this file

In [14]:
df = pd.read_csv(os.path.join(valdir, "6_5_cohort3.csv"))
print(len(df)) # 16704
df.head(5)

16704


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP,Pulse,RR,SBP,Temp
0,JC1000116,131295313275,57868578,2020-09-29 22:45:00+00:00,0,77,81,18,120,36.80
1,JC1000939,131295018112,57698292,2020-08-22 11:41:00+00:00,1,83,112,28,131,38.10
2,JC1001688,131288774622,53810016,2020-06-19 21:36:00+00:00,0,79,114,20,115,36.85
3,JC1001688,131302440087,62198093,2021-02-26 11:46:00+00:00,0,106,120,20,179,37.10
4,JC1001842,131305861173,64314745,2021-02-23 08:08:00+00:00,0,53,70,16,113,36.00


In [15]:
# %load_ext google.cloud.bigquery
# cohort file
table_schema = [{'name' : 'jc_uid', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'inpatient_data_id_coded', 'type': 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'label', 'type' : 'INTEGER'},
                {'name' : 'DBP', 'type' : 'INTEGER'},
                {'name' : 'SBP', 'type' : 'INTEGER'},
                {'name' : 'Pulse', 'type' : 'INTEGER'},
                {'name' : 'RR', 'type' : 'INTEGER'},
                {'name' : 'Temp', 'type' : 'FLOAT'}]
                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_5_cohort3'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          table_schema=table_schema,
          if_exists='replace')

1it [00:02,  2.96s/it]


### Use updated 6_3_cohort with reduced observation to query labs

In [16]:
# m6_labs.sql

q = """ 
SELECT cohort.*,
        order_id_coded, lab_name, base_name, ord_value, ord_num_value, 
        reference_low, reference_high, reference_unit, result_in_range_yn, result_flag, 
        result_time_utc, order_time_utc , taken_time_utc
      
FROM `som-nero-phi-jonc101.shc_core_2021.lab_result` as labs
RIGHT JOIN `som-nero-phi-jonc101.triageTD.6_5_cohort3` as cohort  -- # join labs to cohort

ON labs.pat_enc_csn_id_coded = cohort.pat_enc_csn_id_coded
AND labs.anon_id = cohort.anon_id

WHERE admit_time >= result_time_utc  -- # only labs before admit time
AND extract(year from admit_time) > 2014  -- # only CSNs after 2014
AND base_name in 
    ('AG', 'AGAP', 'BASOAB', 'BUN', 'CL', 'CR', 'EGFR', 'EOSAB', 'GLU', 'HCO3', 'HCO3A', 'HCO3V', 
 'HCT', 'HGB', 'INR', 'K', 'LAC', 'LACWBL', 'LYMAB', 'MONOAB', 'NEUTAB', 'NEUTABS', 'O2SATA', 
 'O2SATV', 'PCAGP', 'PCBUN', 'PCCL', 'PCO2A', 'PCO2V', 'PH', 'PHA', 'PHV', 'PLT', 'PO2A', 'PO2V',
 'PT', 'TBIL', 'TCO2A', 'TNI', 'WBC', 'NA', 'ALB', 'ALKP', 'ALT', 'AST', 'BE', 'CA', 'CO2', 
 'GLOB', 'MCH', 'RDW', 'TP') -- 'GLUURN' removed, all NA
 
"""

query_job=client.query(q)
df=query_job.to_dataframe().to_csv(os.path.join(datadir6, 'labs_2021.csv'), index=False)

### Tiffany's ADT table for cohort4 with labels

In [18]:
q = """
SELECT adt.anon_id, adt.pat_enc_csn_id_coded, adt.effective_time_jittered_utc, adt.seq_num_in_enc,
        adt.pat_class, adt.base_pat_class_c, adt.pat_lvl_of_care_c, adt.pat_lv_of_care, 
        adt.event_type, adt.pat_service
    
FROM shc_core_2021.adt adt
RIGHT JOIN triageTD.6_5_cohort3 c 
ON adt.anon_id = c.anon_id and adt.pat_enc_csn_id_coded = c.pat_enc_csn_id_coded
"""
query_job = client.query(q)
adt = query_job.to_dataframe() # not necessary?
# adt.to_csv(cohort_adt_file, index=False)
adt.to_csv(os.path.join(datadir6, 'cohort_6_3_adt.csv'), index=None)

In [19]:
q = """
select anon_id, death_date_jittered
  from shc_core_2021.demographic
  where anon_id
  in (select anon_id from triageTD.6_5_cohort3)
"""

query_job = client.query(q)
df = query_job.to_dataframe() # not necessary?
# adt.to_csv(cohort_adt_file, index=False)
df.to_csv(os.path.join(datadir6, 'cohort_6_3_demo_deaths2021.csv'), index=None)

### Push some final datasets:
- 6_8_0_cohort4 --> 6_8_cohort4 (removing those already in the original cohort)
- simple data (need to rename race.White to race_White, etc.... to push to avoid having a . in colnames)
- full feature values dataset

In [44]:
# from Tiffany's cohort with labels, and after removing csn in the original cohort
df = pd.read_csv(os.path.join(valdir, "6_7_cohort4.csv"))
df["admit_time"] = pd.to_datetime(df["admit_time"]) 
print(df.shape) # 16700 --> 16484
df.head(5)

(16484, 18)


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max,previous_icu_visit
0,JC1000116,131295313275,57868578,2020-09-29 22:45:00+00:00,0,0,0.0,1,0,0,0,0.0,0.0,0,0,0,0,False
1,JC1000939,131295018112,57698292,2020-08-22 11:41:00+00:00,1,1,0.0,1,0,1,1,0.0,0.0,1,0,1,0,False
2,JC1001688,131288774622,53810016,2020-06-19 21:36:00+00:00,0,0,0.0,1,0,0,0,0.0,0.0,0,0,0,0,False
3,JC1001688,131302440087,62198093,2021-02-26 11:46:00+00:00,0,0,0.0,1,0,0,0,0.0,0.0,0,0,0,0,False
4,JC1001842,131305861173,64314745,2021-02-23 08:08:00+00:00,0,0,0.0,1,0,0,0,0.0,0.0,0,0,0,0,False


In [45]:
# %load_ext google.cloud.bigquery
# cohort file NUMERIC for pat csn
table_schema = [{'name' : 'jc_uid', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'NUMERIC'}, # 'INTEGER'
                {'name' : 'inpatient_data_id_coded', 'type': 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'label_max24', 'type' : 'INTEGER'},
                {'name' : 'label_24hr_recent', 'type' : 'INTEGER'},
                {'name' : 'admit_label', 'type' : 'INTEGER'},
                {'name' : 'has_admit_label', 'type' : 'INTEGER'},
                {'name' : 'died_within_24hrs', 'type' : 'INTEGER'},
                {'name' : 'death_24hr_max_label', 'type' : 'INTEGER'},
                {'name' : 'death_24hr_recent_label', 'type' : 'INTEGER'},
                {'name' : 'first_label', 'type' : 'INTEGER'}, 
                {'name' : 'first_label_minutes_since_admit', 'type' : 'INTEGER'},
                {'name' : 'acute_to_critical_label_recent', 'type' : 'INTEGER'},
                {'name' : 'critical_to_acute_label_recent', 'type' : 'INTEGER'},
                {'name' : 'acute_to_critical_label_max', 'type' : 'INTEGER'},
                {'name' : 'critical_to_acute_label_max', 'type' : 'INTEGER'},
                {'name' : 'previous_icu_visit', 'type' : 'BOOLEAN'}]
                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_7_cohort4' # 1_4_cohort
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [00:03,  3.18s/it]


In [46]:
print(list(df.columns))

['anon_id', 'pat_enc_csn_id_coded', 'inpatient_data_id_coded', 'admit_time', 'label_max24', 'label_24hr_recent', 'admit_label', 'has_admit_label', 'died_within_24hrs', 'death_24hr_max_label', 'death_24hr_recent_label', 'first_label', 'first_label_minutes_since_admit', 'acute_to_critical_label_recent', 'critical_to_acute_label_recent', 'acute_to_critical_label_max', 'critical_to_acute_label_max', 'previous_icu_visit']


In [47]:
# complete cohort up to 2021
df = pd.read_csv(os.path.join(valdir, "6_7_cohort4_all.csv"))
df["admit_time"] = pd.to_datetime(df["admit_time"]) 
print(df.shape) # 16700 --> 60464
df.head(5)

(60464, 18)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max,previous_icu_visit
0,JCd97296,131176042095,18290644,2016-02-06 22:31:00+00:00,0,0,NaN,0,0,0,0,0.0,1325.0,0,0,0,0,NaN
1,JCcdc7e1,131064611420,13865299,2015-01-15 21:16:00+00:00,1,1,1.0,1,0,1,1,1.0,0.0,0,0,0,0,NaN
2,JCe3e5f4,131072326078,14296997,2015-01-28 11:12:00+00:00,1,1,1.0,1,0,1,1,1.0,0.0,0,0,0,0,NaN
3,JCdcfce9,131178712824,18633398,2016-03-04 17:01:00+00:00,1,1,1.0,1,0,1,1,1.0,0.0,0,0,0,0,NaN
4,JCdaaaa6,131211945620,22773101,2016-12-07 22:17:00+00:00,0,0,0.0,1,0,0,0,0.0,0.0,0,0,0,0,NaN


In [48]:
# %load_ext google.cloud.bigquery
# cohort file NUMERIC for pat csn
table_schema = [{'name' : 'jc_uid', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'NUMERIC'}, # 'INTEGER'
                {'name' : 'inpatient_data_id_coded', 'type': 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'label_max24', 'type' : 'INTEGER'},
                {'name' : 'label_24hr_recent', 'type' : 'INTEGER'},
                {'name' : 'admit_label', 'type' : 'INTEGER'},
                {'name' : 'has_admit_label', 'type' : 'INTEGER'},
                {'name' : 'died_within_24hrs', 'type' : 'INTEGER'},
                {'name' : 'death_24hr_max_label', 'type' : 'INTEGER'},
                {'name' : 'death_24hr_recent_label', 'type' : 'INTEGER'},
                {'name' : 'first_label', 'type' : 'INTEGER'}, 
                {'name' : 'first_label_minutes_since_admit', 'type' : 'INTEGER'},
                {'name' : 'acute_to_critical_label_recent', 'type' : 'INTEGER'},
                {'name' : 'critical_to_acute_label_recent', 'type' : 'INTEGER'},
                {'name' : 'acute_to_critical_label_max', 'type' : 'INTEGER'},
                {'name' : 'critical_to_acute_label_max', 'type' : 'INTEGER'},
                {'name' : 'previous_icu_visit', 'type' : 'BOOLEAN'}]
                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_7_cohort4_all' # 1_4_cohort
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [00:06,  6.93s/it]


### Push simpledata

In [25]:
# simple data with vitals sum stats
df = pd.read_csv(os.path.join(valdir, "6_8_coh4_simpledata.csv"))
print(df.shape) # 16700
df.iloc[0:4, 0:23]

(16700, 81)


,anon_id,pat_enc_csn_id_coded,first_label,death_24hr_recent_label,ESI_i,delta_ESI,gender,age,insurance,English,Height_i,delta_H,Weight_i,delta_W,race.Asian,race.Black,race.Native.American,race.Other,race.Pacific.Islander,race.Unknown,race.White,DBP_count,DBP_first_val
0,JC1000116,131295313275,0.0,0,3,0,1,44,1,0,154.00,0,73.05,0,0,0,0,1,0,0,0,5,77
1,JC1000939,131295018112,0.0,1,2,0,1,49,1,0,156.00,0,134.00,0,0,0,0,1,0,0,0,3,83
2,JC1001688,131288774622,0.0,0,3,0,0,37,0,1,206.00,0,163.00,0,1,0,0,0,0,0,0,2,79
3,JC1001688,131302440087,0.0,0,3,0,0,37,0,1,205.74,0,163.29,0,1,0,0,0,0,0,0,2,106


In [26]:
df.columns = df.columns.str.replace(".", "_")
df.iloc[0:4, 0:23]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


,anon_id,pat_enc_csn_id_coded,first_label,death_24hr_recent_label,ESI_i,delta_ESI,gender,age,insurance,English,Height_i,delta_H,Weight_i,delta_W,race_Asian,race_Black,race_Native_American,race_Other,race_Pacific_Islander,race_Unknown,race_White,DBP_count,DBP_first_val
0,JC1000116,131295313275,0.0,0,3,0,1,44,1,0,154.00,0,73.05,0,0,0,0,1,0,0,0,5,77
1,JC1000939,131295018112,0.0,1,2,0,1,49,1,0,156.00,0,134.00,0,0,0,0,1,0,0,0,3,83
2,JC1001688,131288774622,0.0,0,3,0,0,37,0,1,206.00,0,163.00,0,1,0,0,0,0,0,0,2,79
3,JC1001688,131302440087,0.0,0,3,0,0,37,0,1,205.74,0,163.29,0,1,0,0,0,0,0,0,2,106


In [27]:
# can't push if colnames have a dot (from one-hot coding) --> need to rename to push             
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_8_coh4_simpledata'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [00:11, 11.45s/it]


In [ ]:
### Complex data

In [29]:
# final cohort data with all feature values, warning due to time NA for demos
# will be use with feature counts
df = pd.read_csv(os.path.join(valdir, "6_9_coh4_feature_values.csv"))
print(len(df)) # 1190477
df.head(5)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1190477


,anon_id,pat_enc_csn_id_coded,admit_time,first_label,death_24hr_recent_label,death_24hr_max_label,feature_type,features,values,time
0,JC1000116,131295313275,2020-09-29 22:45:00+00:00,0.0,0,0,demo,ESI_i,3.0,NaN
1,JC1000939,131295018112,2020-08-22 11:41:00+00:00,0.0,1,1,demo,ESI_i,2.0,NaN
2,JC1001688,131288774622,2020-06-19 21:36:00+00:00,0.0,0,0,demo,ESI_i,3.0,NaN
3,JC1001688,131302440087,2021-02-26 11:46:00+00:00,0.0,0,0,demo,ESI_i,3.0,NaN
4,JC1001842,131305861173,2021-02-23 08:08:00+00:00,0.0,0,0,demo,ESI_i,3.0,NaN


In [30]:
table_schema = [{'name' : 'anon_id', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'first_label', 'type' : 'INTEGER'},
                {'name' : 'death_24hr_recent_label', 'type' : 'INTEGER'},
                {'name' : 'feature_type', 'type' : 'STRING'},
                {'name' : 'features', 'type' : 'STRING'},
                {'name' : 'values', 'type' : 'FLOAT'},
                {'name' : 'recorded_time', 'type' : 'TIMESTAMP'}]

                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_9_coh4_feature_values'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101',
          table_schema=table_schema,
          if_exists='replace')

1it [00:33, 33.63s/it]


### Feature order counts 
- This is needed for 6.11 notebook

In [49]:
q = open('../SQL/feature_counts/Code_Counts_val.sql', 'r')
query_job =  client.query(q.read()).to_dataframe().to_csv(os.path.join(datadir6, 'coh4_order_code_counts_2021.csv'), index=False)

In [50]:
df = pd.read_csv(os.path.join(datadir6, "coh4_order_code_counts_2021.csv"))
print(len(df)) # 6086852
df.head(5)

2792909


,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
0,JC641497,131284412456,2020-04-01 17:34:00+00:00,Diagnosis,J10.00,1
1,JC641497,131284412456,2020-04-01 17:34:00+00:00,Lab,TROPONIN I,4
2,JC641497,131284412456,2020-04-01 17:34:00+00:00,Diagnosis,Z79.01,1
3,JC641497,131284412456,2020-04-01 17:34:00+00:00,Diagnosis,N39.0,1
4,JC641497,131284412456,2020-04-01 17:34:00+00:00,Diagnosis,I50.43,1


In [51]:
table_schema = [{'name' : 'anon_id', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'feature_type', 'type' : 'STRING'},
                {'name' : 'features', 'type' : 'STRING'},
                {'name' : 'values', 'type' : 'FLOAT'}]

                       
DATASET_NAME = 'triageTD'
TABLE_NAME = 'coh4_order_code_counts_2021'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101',
          table_schema=table_schema,
          if_exists='replace')

1it [01:16, 76.32s/it]


### Push 6_11_coh4_all_features_all_long_year (ALL old and new cohort, ALL features, train and test bins)
This is the data set contains demos, vital and lab counts in bins, and order counts -- long format for modeling
- all new and old cohort of 60,464 unique csn: `6_11_coh4_all_features_all_long_year`

In [53]:
# this one has trainbin and testbin in the same dataframe
# df0 = pd.read_csv(os.path.join(featuredir, "2_9_coh5_features_all_long_year.csv"))
df = pd.read_csv(os.path.join(valdir, "6_11_coh4_all_features_all_long_year.csv"))
print(len(df)) # 14,269,242
df.head(5)

14269242


,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values,year
0,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,demo,ESI_i,3.0,2019
1,JC29f8ad3,131278291027,2019-10-05 23:48:00+00:00,demo,ESI_i,3.0,2019
2,JC29f8b9c,131266787806,2019-05-05 01:07:00+00:00,demo,ESI_i,2.0,2019
3,JC29f8beb,131264387263,2019-03-15 03:35:00+00:00,demo,ESI_i,3.0,2019
4,JC29f8beb,131279241689,2019-11-27 15:29:00+00:00,demo,ESI_i,3.0,2019


In [54]:
df["admit_time"] = pd.to_datetime(df["admit_time"])
df['year'] = df['admit_time'].dt.year

In [55]:
table_schema = [{'name' : 'anon_id', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'feature_type', 'type' : 'STRING'},
                {'name' : 'features', 'type' : 'STRING'},
                {'name' : 'values', 'type' : 'FLOAT'},
                {'name' : 'year', 'type': 'INTEGER'}]

                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_11_coh4_all_features_all_long_year'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101',
          table_schema=table_schema,
          if_exists='replace')

1it [06:50, 410.37s/it]


### Push 1_4_cohort test (2019 and 2020) with predicted results

In [7]:
df = pd.read_csv(os.path.join(modeldir4, "1_4_cohort_test_results.csv"))
print(len(df)) # 12418
df.head(5)

12418


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max,adm_year,transfer,pred_death_24hr_max,pred_death_24hr_recent,pred_first,diff0_24,diff0_True,diff24_True,abs_diff0_24
0,JCe8840f,131264906504,34995073,2019-02-14 22:22:00,0,0,0.0,1,0,0,0,0,0,0,0,0,0,2019,0,0.281178,0.209197,0.235676,0.03,0.24,0.21,0.0
1,JCdb7bb2,131280664882,44427056,2019-11-22 08:10:00,1,1,NaN,0,0,1,1,1,123,0,0,0,0,2019,0,0.952692,0.846169,0.946322,0.10,-0.05,-0.15,0.1
2,JCdf010a,131282861801,45762025,2020-03-04 03:38:00,1,1,0.0,1,0,1,1,0,0,1,0,1,0,2020,1,0.767333,0.714786,0.688232,-0.03,0.69,-0.29,0.0
3,JC2a0fefa,131264837675,34946537,2019-03-01 18:26:00,1,1,1.0,1,0,1,1,1,0,0,0,0,0,2019,0,0.822907,0.485052,0.654735,0.17,-0.35,-0.51,0.2
4,JCcbc03d,131279238299,43525642,2019-11-08 01:14:00,0,0,0.0,1,0,0,0,0,0,0,0,0,0,2019,0,0.359332,0.348007,0.306411,-0.04,0.31,0.35,0.0


In [8]:
DATASET_NAME = 'triageTD'
TABLE_NAME = '1_4_cohort_test_results'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [00:05,  5.91s/it]


In [4]:
df = pd.read_csv(os.path.join(modeldir4, "1_4_cohort_diff_full_features.csv"))
print(len(df)) # 71107 (modeldir4) vs 69334 modeldir4preadmit
df.tail(5)

71107


,anon_id,pat_enc_csn_id_coded,admit_time,first_label,death_24hr_recent_label,death_24hr_max_label,pred_first,pred_death_24hr_recent,abs_diff0_24,diff0_True,diff24_True,feature_type,features,values,time
71102,JCda7d53,131278811777,2019-12-06 03:25:00,1,1,1,0.26965,0.525167,0.3,-0.73,-0.47,labs,Eos,0.01,2019-12-06 00:48:00+00:00
71103,JCda7d53,131278811777,2019-12-06 03:25:00,1,1,1,0.26965,0.525167,0.3,-0.73,-0.47,labs,Lymp,1.23,2019-12-06 00:48:00+00:00
71104,JCda7d53,131278811777,2019-12-06 03:25:00,1,1,1,0.26965,0.525167,0.3,-0.73,-0.47,labs,Basos,0.03,2019-12-06 00:48:00+00:00
71105,JCda7d53,131278811777,2019-12-06 03:25:00,1,1,1,0.26965,0.525167,0.3,-0.73,-0.47,labs,Mono,0.70,2019-12-06 00:48:00+00:00
71106,JCda7d53,131278811777,2019-12-06 03:25:00,1,1,1,0.26965,0.525167,0.3,-0.73,-0.47,labs,Neut,19.62,2019-12-06 00:48:00+00:00


In [6]:
DATASET_NAME = 'triageTD'
TABLE_NAME = '1_4_cohort_diff_full_features'
# TABLE_NAME = '1_4_cohort_24hrpreadmit_diff_full_features'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [00:09,  9.40s/it]


### Push 1_4_cohort all with full features used for modeling

In [9]:
df = pd.read_csv(os.path.join(featuredir, "1_4_cohort_full_features.csv"), engine='python')
print(len(df)) # 9,171,908 modeldir4 vs modeldir4preadmit
df.head(5)

9171898


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max,feature_type,features,values,time,hr_before_admit
0,JCd97296,131176042095,18290644,2016-02-06 22:31:00,0,0,NaN,0,0,0,0,0,1325,0,0,0,0,Lab,CBC WITH DIFFERENTIAL,2.0,NaN,NaN
1,JCd97296,131176042095,18290644,2016-02-06 22:31:00,0,0,NaN,0,0,0,0,0,1325,0,0,0,0,Lab,AB SCREEN (ASI),2.0,NaN,NaN
2,JCd97296,131176042095,18290644,2016-02-06 22:31:00,0,0,NaN,0,0,0,0,0,1325,0,0,0,0,Lab,"DRUGS OF ABUSE SCREEN, URINE",2.0,NaN,NaN
3,JCd97296,131176042095,18290644,2016-02-06 22:31:00,0,0,NaN,0,0,0,0,0,1325,0,0,0,0,Lab,VOLATILE SCREEN,2.0,NaN,NaN
4,JCd97296,131176042095,18290644,2016-02-06 22:31:00,0,0,NaN,0,0,0,0,0,1325,0,0,0,0,Lab,ABO/RH (ARI),2.0,NaN,NaN


In [ ]:
DATASET_NAME = 'triageTD'
TABLE_NAME = '1_4_cohort_full_features'
# TABLE_NAME = '1_4_cohort_24hrpreadmit_full_features'
df.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
          project_id='som-nero-phi-jonc101', 
          if_exists='replace')

1it [04:21, 261.39s/it]


### THICK DESCRIPTION

In [4]:
q = open('../SQL/all_current_dx.sql', 'r')
query_job =  client.query(q.read()).to_dataframe().to_csv(os.path.join(datadir, '1_4_cohort_all_current_dx.csv'), index=False)

### CHECK ADT

In [10]:
# this one for Tif's label notebook, with ADT information for label processing
q = """
SELECT adt.anon_id, adt.pat_enc_csn_id_coded, adt.effective_time_jittered_utc, adt.seq_num_in_enc,
        adt.pat_class, adt.base_pat_class_c, adt.pat_lvl_of_care_c, adt.pat_lv_of_care, 
        adt.event_type, adt.pat_service
    
FROM shc_core.adt adt
RIGHT JOIN triageTD.1_2_cohort c 
ON adt.anon_id = c.anon_id and adt.pat_enc_csn_id_coded = c.pat_enc_csn_id_coded
"""
query_job = client.query(q)
adt = query_job.to_dataframe()
print(len(adt)) # 886988

adt.to_csv(os.path.join(datadir, 'cohort_adt.csv'), index=False)

886988


### Check Admission under order_proc datatable

In [7]:
q = """
SELECT o.order_type, o.display_name, o.description, o.order_class, 
       o.order_status, o.ordering_mode, o.order_time_jittered_utc, c.*
FROM shc_core.order_proc o
RIGHT JOIN triageTD.1_4_cohort c 
ON o.anon_id = c.anon_id and o.pat_enc_csn_id_coded = c.pat_enc_csn_id_coded
WHERE o.order_type = "Admission"
"""
query_job = client.query(q)
ordproc = query_job.to_dataframe()
print(len(ordproc)) #  all 17646270 takes a long time, only admission 112576

112576


In [8]:
ordproc.to_csv(os.path.join(datadir, '1_4_cohort_orderproc_adm.csv'), index=False)

### Push to BQ other files

In [17]:
df = pd.read_csv(os.path.join(featuredir, "2_4_coh3_imputedHWESI.csv"))
print(len(df)) # 45796
print(list(df.columns))
df.head(5)

44258
['anon_id', 'pat_enc_csn_id_coded', 'inpatient_data_id_coded', 'admit_time', 'label', 'ESI_i', 'delta_ESI', 'gender', 'age', 'insurance', 'English', 'Height_i', 'delta_H', 'Weight_i', 'delta_W', 'race.Asian', 'race.Black', 'race.Native.American', 'race.Other', 'race.Pacific.Islander', 'race.Unknown', 'race.White']


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,ESI_i,delta_ESI,gender,age,insurance,...,delta_H,Weight_i,delta_W,race.Asian,race.Black,race.Native.American,race.Other,race.Pacific.Islander,race.Unknown,race.White
0,JC29f8ad2,131274729058,40679773,2019-08-31 12:52:00,0,3,0,1,52,1,...,0,81.0,0,0,0,0,0,0,0,1
1,JC29f8ad3,131278291027,42992239,2019-10-05 23:48:00,0,3,0,1,35,1,...,1,71.0,1,1,0,0,0,0,0,0
2,JC29f8b9c,131266787806,36261582,2019-05-05 01:07:00,0,2,0,0,59,0,...,1,58.0,1,0,0,0,0,0,1,0
3,JC29f8beb,131264387263,34626013,2019-03-15 03:35:00,0,3,0,1,30,1,...,0,65.4,0,0,0,0,0,0,0,1
4,JC29f8beb,131279241689,43527040,2019-11-27 15:29:00,0,3,0,1,30,1,...,0,63.0,0,0,0,0,0,0,0,1


In [12]:
# demographics file
table_schema = [{'name' : 'jc_uid', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'inpatient_data_id_coded', 'type': 'INTEGER'},
                {'name' : 'admit_time', 'type' : 'TIMESTAMP'},
                {'name' : 'label', 'type' : 'INTEGER'},
                {'name' : 'gender', 'type' : 'STRING'},
                {'name' : 'race', 'type' : 'STRING'},
                {'name' : 'language', 'type' : 'STRING'},
                {'name' : 'insurance', 'type' : 'STRING'},
                {'name' : 'recent_height', 'type' : 'INTEGER'},
                {'name' : 'recent_weight', 'type' : 'INTEGER'}, 
                {'name' : 'age', 'type' : 'INTEGER'},
                {'name' : 'recent_date', 'type' : 'DATE'}]
                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '2_4_coh3_imputedHWESI'
cohort_demo.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
                 project_id='som-nero-phi-jonc101',
                 table_schema=table_schema,
                 if_exists='replace')

1it [00:12, 12.14s/it]
